## Minerva Grade Opportunities
### Nicolas A Gort Freitas

In [157]:
import requests
import json
import urllib.parse, urllib.request, urllib.error
import numpy as np
import pandas as pd
import time

np.set_printoptions(suppress=True)
token = '****************************************'
weight_assignment = 12

In [158]:
def retrieve_parallel(key,key2,var): #find keys at the same level in nested dictionaries
    if hasattr(var,'items'):
        for k, v in var.items():
            if k == key:
                try: 
                    yield var[key2]
                except: 
                    pass
                
            if isinstance(v, dict):
                for result in retrieve_parallel(key,key2, v):
                    if type(result)==int or type(result)==str:
                        yield result
            elif isinstance(v, list):
                for d in v:
                    for result in retrieve_parallel(key,key2, d):
                        yield result

In [159]:
lo = requests.get('https://seminar.minerva.kgi.edu/api/v1/hc-index-items?outcomeType=lo',
                headers={'Authorization':'Token '+token}).json()

In [160]:
hc = requests.get('https://seminar.minerva.kgi.edu/api/v1/hc-index-items',
                headers={'Authorization':'Token '+token}).json()

In [161]:
hctree = requests.get('https://seminar.minerva.kgi.edu/api/v1/hc-trees/current?tree',
                headers={'Authorization':'Token '+token}).json()

In [163]:
start = time.clock()


hc_index = {}
hc_indices = list(retrieve_parallel('hashtag','id',hctree))
hc_names = list(retrieve_parallel('id','hashtag',hctree))

for i,j in zip(hc_indices,hc_names):
    hc_index[i]=j
    
print(time.clock() - start)

0.003015000000004875


In [164]:
def logrades(n):
    return requests.get('https://seminar.minerva.kgi.edu/api/v1/outcome-assessments-with-assessed?learning-outcome='+str(n),
                headers={'Authorization':'Token '+token}).json()
def hcgrades(n):
    return requests.get('https://seminar.minerva.kgi.edu/api/v1/outcome-assessments-with-assessed?hc-item='+str(n),
                headers={'Authorization':'Token '+token}).json()


In [165]:
def hc_numgrades(hcid):
    hcvar = hcgrades(hcid)
    weights = []
    for i,j in enumerate(hcvar):
        if j['score']==0.0:
            weights.append(0.0)
        elif j['assignment']==None:
            weights.append(1.0)
        elif 'weight' in j['assignment'].keys():
            weights.append(j['assignment']['weight'])
        else:
            weights.append(weights[-1])

    return sum(weights)

In [166]:
def hc_mean(hcid):
    for i in hc:
        if i['hc-item']==int(hcid):
            if i['mean']==0.0:
                return np.nan
            else:
                return i['mean']

In [167]:
def impact4(x,weight=weight_assignment):
    hcimpact = ((x.Mean*x.N+4*weight)/(x.N+weight))
    if x.N>0:
        return (x.Mean_course*x.HC_count - x.Mean + hcimpact)/(x.HC_count)-x.Mean_course
    if x.N==0:
        #print ('a')
        return (x.Mean_course*x.HC_count+4)/(x.HC_count + 1) - x.Mean_course
    
def impact3(x,weight=weight_assignment):
    hcimpact = ((x.Mean*x.N+3*weight)/(x.N+weight))
    if x.N>0:
        return (x.Mean_course*x.HC_count - x.Mean + hcimpact)/(x.HC_count)-x.Mean_course
    if x.N==0:
        #print ('a')
        return (x.Mean_course*x.HC_count+3)/(x.HC_count + 1) - x.Mean_course

def expected_impact(x,weight=weight_assignment,p=0.5):
    return p*impact4(x)+(1-p)*impact3(x)

In [168]:
start = time.clock()
means = np.array([hc_mean(x) for x in hc_index.keys()])
ngrades = np.array([hc_numgrades(x) for x in hc_index.keys()])
print(time.clock() - start)

1.8457909999999984


In [169]:
cournerstones = list(retrieve_parallel('hashtag','cornerstone-code',hctree))

In [170]:
hc_data = pd.DataFrame({
'HC' : list(hc_index.values()), 
    'Mean' : means,
    'N' : ngrades,
    'Course' : cournerstones
    #'potential4' : potential4,
    #'potential3' : potential3
})

In [171]:
hc_data.head(2)

,Course,HC,Mean,N
0,MC,#openmind,3.75,4.0
1,MC,#critique,3.40,35.0


In [172]:
scores = hc_data.loc[:,['Mean','Course']].groupby('Course').agg({np.mean,lambda x: len(x[x > 0])})
scores
#scores.columns = ['HC_count','Mean']
if scores.iloc[0,0]<=5.0:
    scores.columns = ['Mean','HC_count']
else:
    scores.columns = ['HC_count','Mean']
#MultiIndex(levels=[['Mean'], ['mean', '<lambda>']],
#           labels=[[0, 0], [0, 1]])
scores


,Mean,HC_count
Course,,
CS,3.348096,26.0
EA,3.573762,25.0
FA,3.370950,21.0
MC,3.504877,23.0


In [173]:
joint = hc_data.join(scores,on='Course',rsuffix='_course')

In [174]:
hc_data['Impact_4']=joint.apply(impact4,axis=1)
hc_data['Impact_3']=joint.apply(impact3,axis=1)
hc_data['Expected_impact']=joint.apply(expected_impact,axis=1)

#example: What HCs are most convenient to tag to improve my score in FA?
hc_data.sort_values('Expected_impact',ascending=False)[hc_data.Course=='FA']
#hc_data.sort_values('Impact_4')

/Users/nico/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,Course,HC,Mean,N,Impact_4,Impact_3,Expected_impact
30,FA,#risk,3.000000,7.0,0.030075,0.000000,0.015038
32,FA,#decisiontrees,3.000000,8.0,0.028571,0.000000,0.014286
31,FA,#broadframing,3.166667,6.0,0.026455,-0.005291,0.010582
36,FA,#gametheory,3.200000,5.0,0.026891,-0.006723,0.010084
29,FA,#discounting,3.222222,9.0,0.021164,-0.006047,0.007559
21,FA,#significance,3.040000,25.0,0.014826,-0.000618,0.007104
56,FA,#optimization,3.285714,7.0,0.021482,-0.008593,0.006445
14,FA,#estimation,3.277778,9.0,0.019652,-0.007559,0.006047
16,FA,#distributions,3.181818,22.0,0.013751,-0.003056,0.005348
18,FA,#confidenceintervals,3.300000,10.0,0.018182,-0.007792,0.005195
